In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/gmihaila/ml_things.git
!pip install trl
!pip install transformers[torch]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 16.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_

In [2]:
from dataclasses import dataclass, field
from typing import Optional, Tuple

import torch
from datasets import Dataset, load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser, TrainingArguments

from trl import KTOTrainer, KTOConfig, ModelConfig

In [3]:
# Define and parse arguments.
@dataclass
class ScriptArguments:
    # training parameters
    model_name_or_path: Optional[str] = field(
        default="gpt2", metadata={"help": "the model name"})

    sanity_check: Optional[bool] = field(default=True, metadata={"help": "only train in 1000 samples"})
    # traindata parameters
    train_data: Optional[str] = field(
        default="xx", metadata={"help": "Training data location"})

# When parsing a table with bool values, I get a silent conversion to false for values that can't logically be parsed
def get_data(train_data_path: str, silent: bool = False, cache_dir: str = None) -> Tuple[Dataset, Dataset]:
    datasetall = load_dataset(
        "json",
        data_files={
            train_data_path
        },
        cache_dir=cache_dir,
    )

    def split_prompt_and_responses(sample):
        prompt = sample["prompt"]
        completion = sample["completion"]
        label = sample["label"]
        return {
            "prompt": prompt,
            "completion": completion,
            "label": label,
        }
    datasetall = datasetall.map(split_prompt_and_responses)
    train_test_split = datasetall["train"].train_test_split(test_size=0.8)
    dataset_train = train_test_split['test']
    dataset_test = train_test_split['train']

    return dataset_train, dataset_test


In [4]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

script_args = ScriptArguments(
        model_name_or_path="gpt2",
        sanity_check=True,
        train_data="train_data.json"
    )

# Load the pretrained GPT-2 model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
        script_args.model_name_or_path,
        torch_dtype=torch.float32
    )

tokenizer = AutoTokenizer.from_pretrained(
        script_args.model_name_or_path
    )
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
import pandas as pd
import json

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("../reviews.csv", delimiter='\t', quoting=3)

# Convert DataFrame to list of dictionaries in the desired JSON format
data = []
for index, row in df.iterrows():
    prompt = row['Review']
    completion = str(row['Liked'])
    label = int(completion) == 1
    data.append({
        "prompt": prompt,
        "completion": completion,
        "label": label
    })

# Write the JSON data to a file
with open(script_args.train_data, 'w') as jsonfile:
    json.dump(data, jsonfile, indent=4)


In [6]:
# kto_dataset_dict = {
#     "prompt": [
#         "The food was amazing!",
#         "The food was amazing!",
#         "Excellent service and delicious food.",
#         "Highly recommended, will definitely come back.",
#         "Great atmosphere and friendly staff.",
#         "The best restaurant in town!",
#         "Terrible experience, will not come back.",
#         "Poor service and low-quality food.",
#         "Disappointing meal, wouldn't recommend.",
#         "Unpleasant atmosphere and rude staff.",
#         "Unpleasant atmosphere and rude staff.",
#         "Waste of money, avoid this place.",
#         "Waste of money, avoid this place.",
#     ],
#     "completion": [
#         '1',
#         '0',
#         '1',
#         '1',
#         '1',
#         '1',
#         '0',
#         '0',
#         '0',
#         '0',
#         '1',
#         '0',
#         '1',
#     ],
#     "label": [
#         True,
#         False,
#         True,
#         True,
#         True,
#         True,
#         True,
#         True,
#         True,
#         True,
#         False,
#         True,
#         False,
#     ],
# }

In [7]:
# from datasets import load_dataset
# import json

# dataset_rows = []
# for i in range(len(kto_dataset_dict["prompt"])):
#     row = {
#         "prompt": kto_dataset_dict["prompt"][i],
#         "completion": kto_dataset_dict["completion"][i],
#         "label": kto_dataset_dict["label"][i]
#     }
#     dataset_rows.append(row)

# # Save the list of dictionaries as a JSON array in a file
# with open('train_data.json', 'w') as f:
#     json.dump(dataset_rows, f, indent=4)

# # Load the dataset from the JSON file
# train_dataset = load_dataset('json', data_files='train_data.json')

In [8]:
# Load train and evaluation datasets
train_dataset, eval_dataset = get_data(train_data_path=script_args.train_data)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['prompt', 'completion', 'label'],
    num_rows: 800
})
Dataset({
    features: ['prompt', 'completion', 'label'],
    num_rows: 200
})


In [12]:
kto_args = KTOConfig(
    output_dir = "../content/output_dir",
    beta=0.1,
    desirable_weight=0.25,
    undesirable_weight=1.0
    )
kto_trainer = KTOTrainer(
        model,
        args=kto_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )

kto_trainer_args = kto_trainer.args
kto_trainer_args.fp16 = True  # Use mixed-precision training for faster training

/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:417: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the KTOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:427: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the KTOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:457: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your KTOConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Extracting KL train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Processing tokenized train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Processing tokenized train KL dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Processing tokenized eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Processing tokenized eval KL dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Filtering desirable examples:   0%|          | 0/800 [00:00<?, ? examples/s]

Filtering undesirable examples:   0%|          | 0/800 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:606: UserWarning: 
                        You have different amounts of desirable/positive and undesirable/negative examples but the
                        weights on the desirable and undesirable losses don't seem to be in an ideal range. Based
                        on your data, we recommend EITHER desirable_weight in [1.02, 1.35]
                        or undesirable_weight in [0.19, 0.25] (but NOT BOTH).
                        See the documentation on how to optimally set these weights.
  warnings.warn(


In [13]:
# Fine-tune the GPT-2 model
kto_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss


TrainOutput(global_step=300, training_loss=0.024324979782104492, metrics={'train_runtime': 73.7742, 'train_samples_per_second': 32.532, 'train_steps_per_second': 4.066, 'total_flos': 0.0, 'train_loss': 0.024324979782104492, 'epoch': 3.0})

In [21]:
predictions = kto_trainer.evaluate()
pred = pd.DataFrame(predictions.items(), columns=['Metric', 'Value'])
print(pred)

                     Metric       Value
0                 eval_loss    0.019281
1              eval_runtime    4.990000
2   eval_samples_per_second   40.080000
3     eval_steps_per_second    5.010000
4       eval/rewards/chosen    1.743563
5         eval/logps/chosen   -0.040329
6     eval/rewards/rejected   -9.956087
7       eval/logps/rejected -116.457112
8      eval/rewards/margins   11.699650
9                   eval/kl    0.000000
10                    epoch    3.000000
